In [ ]:
#@title Conexión a la API del Banco Central:
#conectar a la API del Banco Central
import requests
import pandas as pd
url = "https://si3.bcentral.cl/SieteRestWS/SieteRestWS.ashx?user=123456789&pass=tuPassword&firstdate=2021-01-01&lastdate=2021-01-31&timeseries=F022.TPM.TIN.D001.NO.Z.D&function=GetSeries"
response = requests.get(url)
response = response.json()
response = response["Series"]["Obs"]

df_data = pd.DataFrame(response)
df_data.head()


""


In [5]:
#librerias necesarias

import zeep #Crea la conexión con el Webservice
import pandas as pd #Permite trabajar con Dataframes
from zeep.helpers import serialize_object #Facilita el arreglo de los datos recibidos desde el Webservice
import datetime as dt #Verifica el orden correlativo de las fechas
import numpy as np #Permite trabajar con arreglos de datos
from time import sleep #Permite que el programa espere un determinado período de tiempo antes de hacer una nueva consulta
import sys #Detiene la ejecución en caso de error

In [6]:
#definicion de parametros para extraer.
#extraer los datos de las series de tiempo económicas de la API del Banco Central y almacenarlos.

#Parametros
user = '198078263'
pw = 'tu_contraseña'
fInic = '2021-01-01'
fFin = '2021-01-31'

#Series
#A modo de ejemplo, se consultan 2 series:
# - 'Tasa de política monetaria (TPM) (porcentaje)'
# - 'Tipo de Cambio del Dólar Observado'
# - 'Ahorro Nacional/PIB'
# - 'Índice IPC General'
# - 'Unidad tributaria mensual (UTM)'

series=["F022.TPM.TIN.D001.NO.Z.D","F073.TCO.PRE.Z.D","F033.IND.PPI.N.Z.HIST13.0.A","F074.IPC.IND.Z.EP18.C.M","F073.UTR.PRE.Z.M"] #series
series=[x.upper() for x in series]
print(series)

#Conexión
wsdl = 'https://si3.bcentral.cl/SieteWS/SieteWS.asmx?WSDL'
client = zeep.Client(wsdl=wsdl)
client.service.Login(user, pw)

#Consulta
for serie in series:
    print("Consultando serie: " + serie)
    try:
        response = client.service.GetSeries(user, pw, serie, fInic, fFin)
        response = serialize_object(response)
        response = response['Series']['Obs']
        df_data = pd.DataFrame(response)
        df_data['TIME_PERIOD'] = pd.to_datetime(df_data['TIME_PERIOD'])
        df_data = df_data.sort_values(by=['TIME_PERIOD'])
        df_data = df_data.reset_index(drop=True)
        df_data['TIME_PERIOD'] = df_data['TIME_PERIOD'].dt.strftime('%Y-%m-%d')
        df_data['OBS_VALUE'] = pd.to_numeric(df_data['OBS_VALUE'])
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].replace(np.nan, 0)
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].astype(int)
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].astype(str)
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].str.replace('.',',')
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].str.replace(' ','')
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].str.replace('nan','')
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].str.replace('0,0','')
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].str.replace('0','')
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].str.replace(',','.')
        df_data['OBS_VALUE'] = df_data['OBS_VALUE'].replace('', np.nan)
        df_data = df_data.dropna()
        df_data = df_data.reset_index(drop=True)
        df_data.to_csv(serie + '.csv', index=False)
        print("Serie " + serie + " almacenada")
    except:
        print("Error en serie " + serie)
        print(sys.exc_info()[0])
    sleep(1)



['F022.TPM.TIN.D001.NO.Z.D', 'F073.TCO.PRE.Z.D', 'F033.IND.PPI.N.Z.HIST13.0.A', 'F074.IPC.IND.Z.EP18.C.M', 'F073.UTR.PRE.Z.M']


AttributeError: Service has no operation 'Login'

In [ ]:
#Se revisa si hay un código inválido. Todos debiesen terminar en d, m, t, o a (correspondiente a las frecuencias). En caso de ser inválido, se avisa al usuario y se retira de la lista a consultar
for ser_cod in reversed(series):
  if ser_cod[-1] in ["D","M","T","A"]:
    pass
else:
  print("Serie " + ser_cod + " inexistente. Chequea el código")
  series.remove(ser_cod)

#Se identifican las distintas frecuencias de series que pudieran existir y se clasifican por cada tipo
series_freq=[x[-1] for x in series]
series_freq=list(np.unique(series_freq))

#series_freq será la lista que contendrá los valores únicos de frecuencia
print(series_freq)

#Ahora, se convierte la inicial de la frecuencia a su nombre, que es el que se necesita para hacer la consulta
for x in range(len(series_freq)):
  if series_freq[x]=="D":
    series_freq[x]=series_freq[x].replace("D","DAILY")
  elif series_freq[x]=="M":
    series_freq[x]=series_freq[x].replace("M","MONTHLY")
  elif series_freq[x]=="T":
    series_freq[x]=series_freq[x].replace("T","QUARTERLY")
  elif series_freq[x]=="A":
    series_freq[x]=series_freq[x].replace("A","ANNUAL")
  else:
    pass
print(series_freq)


In [ ]:

#Se identifica la dirección del WSDL (Web Service Definition Language) que permitirá a la librería zeep identificar qué consultas pueden hacerse al Webservice, además de generar el objeto client, que permitirá el intercambio de datos
wsdl="https://si3.bcentral.cl/SieteWS/SieteWS.asmx?wsdl"
client = zeep.Client(wsdl)

#meta_series contendrá los datos recolectados y ordenados obtenidos desde "SearchSeries" para todas las frecuencias
meta_series=pd.DataFrame()

#Se itera dentro de la lista series_freq para consultar las distintas frecuencias de interés:
for frequ in series_freq:
  for attempt in range(4):
    try:
      #Se consulta usando el usuario, password y frecuencia de interés
      res_search=client.service.SearchSeries(user,pw,frequ)

      #Se limpia la información obtenida
      res_search=res_search["SeriesInfos"]["internetSeriesInfo"]
      res_search = serialize_object(res_search)

      #Se crea un diccionario con las series obtenidas y los datos de interés (título, código y frencuencia)
      res_search = { serie_dict['seriesId']:[serie_dict['spanishTitle'],serie_dict['frequency']] for serie_dict in res_search }

      #A partir del diccionario creado, se arma un dataframe (meta_series_aux) que luego se agrega al dataframe que contendrá todas las frecuencias (meta_series)
      meta_series_aux=pd.DataFrame.from_dict(res_search,orient='index')
      meta_series=meta_series.append(meta_series_aux)
      print("Frecuencia " + str(frequ) + " encontrada. Agregando")
      break
    except:
      print("Intento " + str(attempt) + ": La frecuencia " + str(frequ) + " no fue encontrada")
      #En caso de error, se esperan 20 segundos antes de volver a consultar la serie
      sleep(20)
  else:
    print("Frecuencia " + str(frequ) + " no fue encontrada. Deteniendo ejecución")
    sys.exit("Deteniendo ejecución")

#Finalmente, se limpia el Dataframe obtenido para conservar sólo las series de interés:
meta_series=meta_series.loc[series]
meta_series.columns=["spanishTitle","frequency"]
print(meta_series)



In [ ]:

#Creación del DataFrame values_df, que incluirá todas las series que se consultarán
values_df=pd.DataFrame()

#Iteración por cada una de las series consultando los datos. Los valores obtenidos se agregan a values_df
for serieee in series:
  #Se genera un loop para hacer 10 intentos de consulta por serie. Si tiene éxito, continúa con la siguiente serie, si no tiene éxito, intenta nuevamente
  for attempt in range(4):
    try:
      #Creación del objeto que contendrá el código de serie
      ArrayOfString = client.get_type('ns0:ArrayOfString')
      value = ArrayOfString(serieee)

      #Se ejecuta la consulta utilizando los parámetros ingresados (usuario, password, fecha de inicio, fecha final y código de serie) y se asigna a la variable result
      result = client.service.GetSeries(user,pw,fInic,fFin, value)
      #Se omite la serie si no hay observaciones en el período solicitado
      if result["Series"]["fameSeries"][0]["obs"]==[]:
        print("La serie "+ str(serieee) + " no tiene observaciones para el período seleccionado")
        break
      #Se limpia la información obtenida, dejando como nombre de fila el␣
,→código de serie y, como columnas, las fechas en formato dd-mm-aaaa
result = serialize_object(result["Series"]["fameSeries"][0]["obs"])
result=pd.DataFrame(result).T
result.columns=result.iloc[0,:]
result=result.drop(result.index[0:3],axis=0)
result.index=[serieee]
#Se agrega la serie ordenada al DataFrame values_df
values_df=values_df.append(result,sort=True)
print("Serie " + str(serieee) + " encontrada. Agregando")
break
except:
print("Intento " + str(attempt) + ": La serie " + str(serieee) + "␣
,→no fue encontrada")
#En caso de error, se esperan 20 segundos antes de volver a␣
,→consultar la serie
sleep(20)
else:
print("La serie " + str(serieee) + " no fue encontrada. Omitiendo")


In [ ]:
#@title Pruebas de calidad de datos
#definir metricas y criterios para evaluar la calidad de los datos de las series de tiempo.
#implementar pruebas automatizadas basadas en estas metricas y criterios para evaluar la calidad de los datos.
#realizar pruebas de integridad, consistencia, valores atipicos, valores faltantes, inconsistencias temporales, entre otros.

In [ ]:
#@title Generacion del informe de calidad de los datos.


